# STAC API

## A short word on STAC

STAC is the central way of accessing any spatio-temporal data on terrabyte. See here for an introduction and the detailed sepcification:
* https://stacspec.org/en
* https://github.com/radiantearth/stac-spec

Principally, data is offered over a *catalog* containing data from various sources. This catalog is further sub-divided into *collections*. 
A collection could for example contain a certain satellite data product like Sentinel-1 GRD, SLC or Sentinel-2 L2A. 
Each collection consists of multiple items, which might represent individual satellite scenes or product tiles (e.g. the MGRS tiles of Sentinel-2). 
Each item consists of one or many *assets*, which contain links to the actual data. For example individual GeoTIFF files for each band.

The terrabyte STAC catalog URL is https://stac.terrabyte.lrz.de/public/api.

Further down you will find examples on how to list all available collections and how to query the STAC catalog in Python using the pystac library. The content can also be explored by opening the above link in a browser.

![stac-catalog.png](_images/stac-catalog.png)

From there you can navigate to individual collections and explore their metadata content for search queries. For example, under https://stac.terrabyte.lrz.de/public/api/collections/sentinel-2-c1-l2a/items → `features` → `0` → `properties` we can list the properties of the first item of the sentinel-2-c1-l2a collection. This list contains, amongst others, the entries `eo:cloud_cover` and `s2:mgrs_tile`, which will be queried in the tutorial below. The namespaces eo, grid, proj, mgrs, sat and view refer to dedicated STAC extensions. s2 is a custom namespace for expressing Sentinel-2 specific metadata that cannot be described in other STAC extensions.

![stac-collection.png](_images/stac-collection.png)

## Explore the STAC catalog

In [1]:
from pystac_client import Client as pystacclient
catalog = pystacclient.open('https://stac.terrabyte.lrz.de/public/api')
 
# list the IDs of all STAC catalog collections
for collection in catalog.get_all_collections():
    print( collection.id)

sentinel-1-grd
modis-13a2-061
sentinel-2-c1-l1c
sentinel-3-olci-l1-efr
sentinel-2-c1-l2a
viirs-09ga-001
sentinel-2-l1c
modis-10a1-061
cop-dem-glo-90
modis-09gq-061
cop-dem-glo-30
viirs-13a1-001
sentinel-1-slc
landsat-tm-c2-l2
landsat-etm-c2-l2
sentinel-1-nrb
landsat-ot-c2-l2
modis-13q1-061
viirs-15a2h-001
sentinel-2-l2a
modis-13a3-061
modis-09ga-061


## Query data from the STAC catalog

gte and lte describe greater than or equal and lower than or equal, respectively.

In [2]:
query = {
    'eo:cloud_cover': {
        "gte": 0,
        "lte": 20
    }
}

start = '2024-01-01T00:00:00Z'
end = '2024-01-31T23:59:59Z'
collection = 'sentinel-2-l1c'

results = catalog.search(
    collections=[collection],
    datetime=[start, end],
    query=query,
    limit=500
)

items = results.item_collection()
print("%s items found" % len(items))

175 items found


## Show first item

In [3]:
items[0]

<Item id=S2A_MSIL1C_20240131T102251_N0510_R065_T33UVA_20240131T110507>

## List items as Pandas GeoDataFrame

In [4]:
import geopandas as gpd
dataframe = gpd.GeoDataFrame.from_features(items)
dataframe

,geometry,created,datetime,platform,grid:code,proj:epsg,providers,s2:tile_id,instruments,view:azimuth,...,view:sun_azimuth,mgrs:latitude_band,s2:generation_time,sat:relative_orbit,view:sun_elevation,view:incidence_angle,s2:processing_baseline,terrabyte:stactools_id,s2:degraded_msi_data_percentage,s2:reflectance_conversion_factor
0,"POLYGON ((13.43460 55.03692, 13.47192 54.05050...",2024-05-22T22:10:36.728632Z,2024-01-31T10:22:51.024000Z,sentinel-2a,MGRS-33UVA,32633,[{'url': 'https://earth.esa.int/web/guest/home...,S2A_OPER_MSI_L1C_TL_2APS_20240131T110507_A0449...,[msi],282.329641,...,167.384417,U,2024-01-31T11:05:07.000000Z,65,17.180202,5.413319,05.10,S2A_T33UVA_20240131T102307_L1C,0.0224,1.031459
1,"POLYGON ((11.87240 55.00678, 11.94686 54.02143...",2024-05-22T22:10:36.232857Z,2024-01-31T10:22:51.024000Z,sentinel-2a,MGRS-33UUA,32633,[{'url': 'https://earth.esa.int/web/guest/home...,S2A_OPER_MSI_L1C_TL_2APS_20240131T110507_A0449...,[msi],129.390805,...,165.873870,U,2024-01-31T11:05:07.000000Z,65,16.993550,3.420010,05.10,S2A_T33UUA_20240131T102307_L1C,0.0000,1.031459
2,"POLYGON ((10.65222 54.04805, 12.20249 54.01754...",2024-05-22T22:10:35.500744Z,2024-01-31T10:22:51.024000Z,sentinel-2a,MGRS-32UPF,32632,[{'url': 'https://earth.esa.int/web/guest/home...,S2A_OPER_MSI_L1C_TL_2APS_20240131T110507_A0449...,[msi],106.081111,...,164.543498,U,2024-01-31T11:05:07.000000Z,65,16.796344,8.428263,05.10,S2A_T32UPF_20240131T102307_L1C,0.0140,1.031459
3,"POLYGON ((10.51678 53.74951, 10.49538 53.15178...",2024-05-22T22:10:35.443102Z,2024-01-31T10:22:51.024000Z,sentinel-2a,MGRS-32UPE,32632,[{'url': 'https://earth.esa.int/web/guest/home...,S2A_OPER_MSI_L1C_TL_2APS_20240131T110507_A0449...,[msi],106.829456,...,164.478885,U,2024-01-31T11:05:07.000000Z,65,17.663519,7.375773,05.10,S2A_T32UPE_20240131T102307_L1C,0.0086,1.031459
4,"POLYGON ((10.25062 53.15255, 10.64157 53.14986...",2024-05-22T22:10:34.753945Z,2024-01-31T10:22:51.024000Z,sentinel-2a,MGRS-32UNE,32632,[{'url': 'https://earth.esa.int/web/guest/home...,S2A_OPER_MSI_L1C_TL_2APS_20240131T110507_A0449...,[msi],110.784772,...,163.002902,U,2024-01-31T11:05:07.000000Z,65,17.395278,11.160383,05.10,S2A_T32UNE_20240131T102307_L1C,0.0607,1.031459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,"POLYGON ((10.31565 47.00877, 10.31189 46.85818...",2024-05-22T22:42:28.515243Z,2024-01-03T10:13:29.024000Z,sentinel-2b,MGRS-32TPT,32632,[{'url': 'https://earth.esa.int/web/guest/home...,S2B_OPER_MSI_L1C_TL_2BPS_20240103T110304_A0356...,[msi],105.238696,...,164.892849,T,2024-01-03T11:03:04.000000Z,22,18.419362,7.892470,05.10,S2B_T32TPT_20240103T101328_L1C,0.0184,1.034219
171,"POLYGON ((10.26126 46.85777, 10.44015 46.85664...",2024-05-22T22:42:28.474786Z,2024-01-03T10:13:29.024000Z,sentinel-2b,MGRS-32TNT,32632,[{'url': 'https://earth.esa.int/web/guest/home...,S2B_OPER_MSI_L1C_TL_2BPS_20240103T110304_A0356...,[msi],110.138552,...,163.645813,T,2024-01-03T11:03:04.000000Z,22,18.162785,11.487888,05.10,S2B_T32TNT_20240103T101328_L1C,0.0000,1.034219
172,"POLYGON ((12.59705 48.72633, 12.28064 48.72091...",2024-05-22T22:10:07.573827Z,2024-01-01T10:24:31.024000Z,sentinel-2a,MGRS-33UUP,32633,[{'url': 'https://earth.esa.int/web/guest/home...,S2A_OPER_MSI_L1C_TL_2APS_20240101T110513_A0445...,[msi],294.386202,...,169.411024,U,2024-01-01T11:05:13.000000Z,65,18.059015,11.218722,05.10,S2A_T33UUP_20240101T102428_L1C,0.0022,1.034107
173,"POLYGON ((13.00213 49.58197, 11.76852 49.61961...",2024-05-22T22:10:07.445907Z,2024-01-01T10:24:31.024000Z,sentinel-2a,MGRS-32UQV,32632,[{'url': 'https://earth.esa.int/web/guest/home...,S2A_OPER_MSI_L1C_TL_2APS_20240101T110513_A0445...,[msi],288.930685,...,168.884064,U,2024-01-01T11:05:13.000000Z,65,17.135781,8.862945,05.10,S2A_T32UQV_20240101T102428_L1C,0.0323,1.034107


## Visualize the covered area

In [5]:
import folium
import folium.plugins as folium_plugins

map = folium.Map()
layer_control = folium.LayerControl(position='topright', collapsed=True)
fullscreen = folium_plugins.Fullscreen()
style = {'fillColor': '#00000000', "color": "#0000ff", "weight": 1}

footprints = folium.GeoJson(
    dataframe.to_json(),
    name='Stac Item footprints',
    style_function=lambda x: style,
    control=True
)

footprints.add_to(map)
layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds())
map